In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

import numpy as np



In [3]:
! cd /mnt/ssd1/mehul_data/research 

### Load data

In [4]:
location = '/mnt/ssd1/mehul_data/research/'
location = '/mnt/ssd1/mehul_data/research/'
fc_small = np.load(location + 'fc_small_data.npy', allow_pickle=True).item()

final_pearson = fc_small['corr']
labels = fc_small['label']
site = fc_small['site']

final_pearson, labels = [torch.from_numpy(data).float() for data in (final_pearson, labels)]
# site = torch.from_numpy(site)

### Dataloader

In [5]:
BATCH_SIZE = 32

In [14]:
from sklearn.model_selection import StratifiedShuffleSplit

def create_graph_data(final_pearson: torch.Tensor, node_feature: torch.Tensor, labels: torch.Tensor, site: torch.Tensor):
    
    graph_data_list = []
    print(final_pearson.shape[0])
    for i in range(final_pearson.shape[0]):
        edge_index = final_pearson[i].nonzero()(as_tuple=False).t().contiguous()

        data = Data(x=node_feature, edge_index=edge_index, y=labels[i], site=site[i])
        graph_data_list.append(data)

    return graph_data_list

def init_stratified_dataloader(final_pearson: torch.tensor, 
                               labels: torch.tensor,
                               site: torch.tensor,
                               stratified: torch.tensor,):
    
    num_graphs, num_nodes = final_pearson.shape[0], final_pearson.shape[1]
    node_feature = torch.ones((num_graphs, num_nodes))
    graph_data_list = create_graph_data(final_pearson, node_feature, labels, site)

    # Stratified split
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    for train_index, test_index in split.split(final_pearson, stratified):
        train_data_list = [graph_data_list[i] for i in train_index]
        test_data_list = [graph_data_list[i] for i in test_index]

    # create pyg dataloader
    train_dataloader = DataLoader(
        train_data_list, batch_size=BATCH_SIZE, shuffle=True)
    test_dataloader = DataLoader(
        test_data_list, batch_size=BATCH_SIZE, shuffle=False)
    
    return train_dataloader, test_dataloader

In [15]:
# train_dataloader, test_dataloader = init_stratified_dataloader(
#     final_pearson, labels, site, stratified=site)

init_stratified_dataloader(
    final_pearson, labels, site, stratified=site)


205


TypeError: 'Tensor' object is not callable